<a href="https://colab.research.google.com/github/Nituhimani/MLcodebasics/blob/feature%2Fxyz/Action_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os



In [99]:


dataset_path = os.listdir('/content/drive/MyDrive/Dataset/Train')

label_types = os.listdir('/content/drive/MyDrive/Dataset/Train')
print (label_types)

['Listening', 'Speaking', 'Explaining']


In [100]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/content/drive/MyDrive/Dataset/Train' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/content/drive/MyDrive/Dataset/Train' + '/' +item) + '/' + room))

# Build a dataframe
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())

         tag                                         video_name
0  Listening  /content/drive/MyDrive/Dataset/Train/Listening...
1  Listening  /content/drive/MyDrive/Dataset/Train/Listening...
2   Speaking  /content/drive/MyDrive/Dataset/Train/Speaking/...
3   Speaking  /content/drive/MyDrive/Dataset/Train/Speaking/...
4   Speaking  /content/drive/MyDrive/Dataset/Train/Speaking/...
           tag                                         video_name
6   Explaining  /content/drive/MyDrive/Dataset/Train/Explainin...
7   Explaining  /content/drive/MyDrive/Dataset/Train/Explainin...
8   Explaining  /content/drive/MyDrive/Dataset/Train/Explainin...
9   Explaining  /content/drive/MyDrive/Dataset/Train/Explainin...
10  Explaining  /content/drive/MyDrive/Dataset/Train/Explainin...


In [101]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')

Preparing test data



In [102]:
dataset_path = os.listdir('/content/drive/MyDrive/Dataset/Test')
print(dataset_path)

room_types = os.listdir('/content/drive/MyDrive/Dataset/Test')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/content/drive/MyDrive/Dataset/Test' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/content/drive/MyDrive/Dataset/Test' + '/' +item) + '/' + room))

# Build a dataframe
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('test.csv')

['Explaining', 'Listening', 'Speaking']
Types of activities found:  3
          tag                                         video_name
0  Explaining  /content/drive/MyDrive/Dataset/Test/Explaining...
1   Listening  /content/drive/MyDrive/Dataset/Test/Listening/...
2    Speaking  /content/drive/MyDrive/Dataset/Test/Speaking/W...
          tag                                         video_name
0  Explaining  /content/drive/MyDrive/Dataset/Test/Explaining...
1   Listening  /content/drive/MyDrive/Dataset/Test/Listening/...
2    Speaking  /content/drive/MyDrive/Dataset/Test/Speaking/W...


In [103]:
!pip install tensorflow_docs

In [104]:

import cv2
import os
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio

In [105]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

Data preparation


In [106]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 11
Total videos for testing: 3


,Unnamed: 0,video_name,tag
5,5,/content/drive/MyDrive/Dataset/Train/Speaking/...,Speaking
8,8,/content/drive/MyDrive/Dataset/Train/Explainin...,Explaining
2,2,/content/drive/MyDrive/Dataset/Train/Speaking/...,Speaking
7,7,/content/drive/MyDrive/Dataset/Train/Explainin...,Explaining
9,9,/content/drive/MyDrive/Dataset/Train/Explainin...,Explaining
0,0,/content/drive/MyDrive/Dataset/Train/Listening...,Listening
4,4,/content/drive/MyDrive/Dataset/Train/Speaking/...,Speaking
10,10,/content/drive/MyDrive/Dataset/Train/Explainin...,Explaining
6,6,/content/drive/MyDrive/Dataset/Train/Explainin...,Explaining
1,1,/content/drive/MyDrive/Dataset/Train/Listening...,Listening


Feed the videos to a network


In [107]:
# The following two methods are taken from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

Feature extraction

In [108]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

Label encoding

In [109]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['Explaining', 'Listening', 'Speaking']


array([[1],
       [1],
       [2],
       [2],
       [2],
       [2],
       [0],
       [0],
       [0],
       [0],
       [0]])

In [110]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 30
NUM_FEATURES = 2048

In [111]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()

    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["tag"].values

    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")

# MAX_SEQ_LENGTH = 20, NUM_FEATURES = 2048. We have defined this above under hyper parameters


1/1 [==============================] - 0s 246ms/step
Frame features in train set: (11, 30, 2048)
Frame masks in train set: (11, 30)
train_labels in train set: (11, 1)
test_labels in train set: (3, 1)


The sequence model

In [112]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/30
1/1 [==============================] - ETA: 0s - loss: 1.2130 - accuracy: 0.1429
Epoch 1: val_loss improved from inf to 1.01924, saving model to ./tmp/video_classifier
1/1 [==============================] - 11s 11s/step - loss: 1.2130 - accuracy: 0.1429 - val_loss: 1.0192 - val_accuracy: 0.7500
Epoch 2/30
1/1 [==============================] - ETA: 0s - loss: 1.1257 - accuracy: 0.1429
Epoch 2: val_loss did not improve from 1.01924
1/1 [==============================] - 0s 140ms/step - loss: 1.1257 - accuracy: 0.1429 - val_loss: 1.0455 - val_accuracy: 0.7500
Epoch 3/30
1/1 [==============================] - ETA: 0s - loss: 1.0089 - accuracy: 0.2857
Epoch 3: val_loss improved from 1.01924 to 0.97035, saving model to ./tmp/video_classifier
1/1 [==============================] - 0s 156ms/step - loss: 1.0089 - accuracy: 0.2857 - val_loss: 0.9703 - val_accuracy: 1.0000
Epoch 4/30
1/1 [==============================] - ETA: 0s - loss: 1.0939 - accuracy: 0.2857
Epoch 4: val_loss did

In [113]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

Test video path: /content/drive/MyDrive/Dataset/Test/Speaking/WhatsApp Video 2023-09-25 at 13.48.29.mp4
1/1 [==============================] - 3s 3s/step
  Explaining: 39.19%
  Speaking: 30.61%
  Listening: 30.20%


In [114]:
!pip install moviepy

In [117]:
from moviepy.editor import *


Inference

In [118]:
def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(video_path)
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]
    return probabilities

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

#test_video = np.random.choice(test_df["video_name"].values.tolist())
#print(f"Test video path: {test_video}")


#test_frames = sequence_prediction(test_video)

In [123]:
clip = VideoFileClip("/content/WhatsApp Video 2023-09-25 at 15.10.13.mp4")
start=0
end=1
l=[]
action=["Explaining","Speaking","Listening"]

for i in range(3):
  try:
    clip1 = clip.subclip(start,end)
    clip1.write_videofile('edited.mp4',codec='libx264')
    test_frames = sequence_prediction("edited.mp4")
    predicted_action=action[np.argmax(test_frames)]
    l.append(predicted_action)
    start+=1
    end+=1
  except:
    pass



Moviepy - Building video edited.mp4.
MoviePy - Writing audio in editedTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video edited.mp4



Moviepy - Done !
Moviepy - video ready edited.mp4
1/1 [==============================] - 0s 41ms/step
Moviepy - Building video edited.mp4.
MoviePy - Writing audio in editedTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video edited.mp4



Moviepy - Done !
Moviepy - video ready edited.mp4
1/1 [==============================] - 0s 29ms/step
Moviepy - Building video edited.mp4.
MoviePy - Writing audio in editedTEMP_MPY_wvf_snd.mp3


In [124]:
l1 = list(range(len(l)))
pd.DataFrame([l1,l]).T

,0,1
0,0,Explaining
1,1,Explaining
